In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-01-01 14:42:04--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M   157MB/s    in 0.2s    

2024-01-01 14:42:04 (157 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Filter out users with less than 200 ratings
user_ratings_count = df['user'].value_counts()
active_users = user_ratings_count[user_ratings_count >= 200].index
df_filtered_users = df[df['user'].isin(active_users)]


In [ ]:
book_ratings_count = df_filtered_users['isbn'].value_counts()
popular_books = book_ratings_count[book_ratings_count >= 100].index
df_filtered = df_filtered_users[df_filtered_users['isbn'].isin(popular_books)]


In [ ]:
book_ratings_pivot = df_filtered.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

# Convert the pivot table to a sparse matrix
book_ratings_csr = csr_matrix(book_ratings_pivot.values)

# Build the Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
model_knn.fit(book_ratings_csr)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [ ]:
def get_recommends(book_title, book_ratings_pivot, model_knn):
    # Find the index of the given book in the filtered dataframe using fuzzy matching
    matches = process.extractOne(book_title, df_filtered['title'], scorer=fuzz.partial_ratio)

    if matches and matches[1] >= 80:
        book_title_matched = matches[0]

        # Check if the book title exists in the filtered dataframe
        if book_title_matched in df_filtered['title'].values:
            # Get the index of the book in the filtered dataframe
            book_index_df = df_filtered[df_filtered['title'] == book_title_matched].index[0]

            # Find the corresponding index in the book_ratings_pivot DataFrame
            try:
                book_index_pivot = book_ratings_pivot.index.get_loc(df_filtered['isbn'].iloc[book_index_df])
            except KeyError:
                return [f"Index not found for book '{book_title_matched}' in the book_ratings_pivot DataFrame."]

            # Use the Nearest Neighbors model to get recommendations
            distances, indices = model_knn.kneighbors(book_ratings_pivot.iloc[book_index_pivot, :].values.reshape(1, -1), n_neighbors=6)

            recommended_books = []
            for i in range(1, len(distances.flatten())):
                recommended_books.append([df_filtered['title'].iloc[indices.flatten()[i]], distances.flatten()[i]])

            return [book_title_matched, recommended_books]
        else:
            return [f"Book '{book_title_matched}' not found in the dataset."]
    else:
        return [f"No close match found for the book '{book_title}'. Please check the spelling or try another book."]

# Test the function
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))", book_ratings_pivot, model_knn)

["No close match found for the book 'The Queen of the Damned (Vampire Chronicles (Paperback))'. Please check the spelling or try another book."]

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

TypeError: ignored